In [2]:
import pickle
import json
# import seaborn as sns
import matplotlib.transforms as mtransforms
from bread import vis
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bread.algo import tracking
from bread.data import *
from sklearn.metrics import confusion_matrix
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
from glob import glob
import sys
import os.path
sys.path.append('../../src/libraries')  # import YeaZ

In [3]:
import math

def calculate_growth_rate(initial_count, final_count, time_interval):
    growth_rate = math.log(final_count / initial_count) / time_interval
    return growth_rate


## for our data

In [4]:
tot_edges = 0
tot_true_edges = 0
info_df = pd.DataFrame(columns=['colony', 'num_frames', 'start_cells', 'end_cells', 'total_cells', 'total_true', 'total_edges', 'tot_to_true_ratio'])
for i in [0,1,2,3,4,5,6,7,8]:
    seg = SegmentationFile.from_h5(
    f'../../data/segmentations/colony00{i}_segmentation.h5').get_segmentation("FOV0")
    num_frames = len(seg)
    start_cells = len(seg.cell_ids(0))
    end_cells = len(seg.cell_ids(num_frames-1))
    total_cells = end_cells
    total_true = 0
    total_edges = 0

    # microscopy = Microscopy.from_tiff(
    #     '../../data/microscopies/colony000_microscopy.tif')
    # feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1)
    for t in range(len(seg)-1):
        n_cells_t1 = len(seg.cell_ids(t))
        n_cells_t2 = len(seg.cell_ids(t+1))
        total_cells += n_cells_t1
        total_edges+= n_cells_t1*n_cells_t2
        total_true += n_cells_t2
    tot_to_true_ratio = total_edges/total_true
    row = {
        'colony': i,
        'num_frames': num_frames,
        'start_cells': start_cells,
        'end_cells': end_cells,
        'total_cells': total_cells,
        'total_true': total_true,
        'total_edges': total_edges,
        'tot_to_true_ratio': tot_to_true_ratio
    }
    row_df = pd.DataFrame([row])
    info_df = pd.concat([info_df, row_df])

    tot_edges += total_edges
    tot_true_edges += total_true

print(tot_edges/tot_true_edges)
print(tot_edges)
print(tot_true_edges)



168.136416103272
14275118
84902


In [5]:
info_df

,colony,num_frames,start_cells,end_cells,total_cells,total_true,total_edges,tot_to_true_ratio
0,0,146,4,354,11973,11969,2171712,181.444732
0,1,181,2,82,5223,5221,244669,46.862478
0,2,181,7,155,9960,9953,886708,89.089521
0,3,181,2,96,4986,4984,253895,50.942014
0,4,181,2,123,5753,5751,379526,65.993045
0,5,181,2,38,2722,2720,60156,22.116176
0,6,120,3,331,8263,8260,1283611,155.400847
0,7,97,44,357,23683,23639,6649708,281.302424
0,8,145,3,366,12408,12405,2345133,189.047400


# for benchmarking data (from YIT)

In [6]:
tot_edges = 0
tot_true_edges = 0
info_df_bench_mark = pd.DataFrame(columns=['colony', 'num_frames', 'start_cells', 'end_cells', 'total_cells', 'total_true', 'total_edges', 'tot_to_true_ratio'])
for i in [1,2,3,4,5,6,7]:
    seg = SegmentationFile.from_h5(
    f'../../data/YIT/test_set_{i}_segmentation.h5').get_segmentation("FOV0")
    num_frames = len(seg)
    start_cells = len(seg.cell_ids(0))
    end_cells = len(seg.cell_ids(num_frames-1))
    total_cells = end_cells
    total_true = 0
    total_edges = 0

    # microscopy = Microscopy.from_tiff(
    #     '../../data/microscopies/colony000_microscopy.tif')
    # feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1)
    for t in range(len(seg)-1):
        n_cells_t1 = len(seg.cell_ids(t))
        n_cells_t2 = len(seg.cell_ids(t+1))
        total_cells += n_cells_t1
        total_edges+= n_cells_t1*n_cells_t2
        total_true += n_cells_t2
    tot_to_true_ratio = total_edges/total_true
    row = {
        'colony': i,
        'num_frames': num_frames,
        'start_cells': start_cells,
        'end_cells': end_cells,
        'total_cells': total_cells,
        'total_true': total_true,
        'total_edges': total_edges,
        'tot_to_true_ratio': tot_to_true_ratio
    }
    row_df = pd.DataFrame([row])
    info_df_bench_mark = pd.concat([info_df_bench_mark, row_df])

    tot_edges += total_edges
    tot_true_edges += total_true

print(tot_edges/tot_true_edges)
print(tot_edges)
print(tot_true_edges)
info_df_bench_mark


138.88841458208827
1626800
11713


,colony,num_frames,start_cells,end_cells,total_cells,total_true,total_edges,tot_to_true_ratio
0,1,60,14,27,1150,1136,22256,19.591549
0,2,30,4,6,144,140,692,4.942857
0,3,20,101,134,2257,2156,242570,112.509276
0,4,20,163,225,3965,3802,756084,198.864808
0,5,20,135,155,2879,2744,394630,143.815598
0,6,10,34,49,433,399,17187,43.075188
0,7,10,126,169,1462,1336,193381,144.746257


In [7]:
# merge the two dataframes
info_df = pd.concat([info_df, info_df_bench_mark])
info_df = info_df.reset_index(drop=True)

In [10]:
info_df

,colony,num_frames,start_cells,end_cells,total_cells,total_true,total_edges,tot_to_true_ratio
0,0,146,4,354,11973,11969,2171712,181.444732
1,1,181,2,82,5223,5221,244669,46.862478
2,2,181,7,155,9960,9953,886708,89.089521
3,3,181,2,96,4986,4984,253895,50.942014
4,4,181,2,123,5753,5751,379526,65.993045
5,5,181,2,38,2722,2720,60156,22.116176
6,6,120,3,331,8263,8260,1283611,155.400847
7,7,97,44,357,23683,23639,6649708,281.302424
8,8,145,3,366,12408,12405,2345133,189.047400
9,1,60,14,27,1150,1136,22256,19.591549


In [12]:
info_df['source'] = [['ours-crowded']+['ours']*5 + ['external']+['marco']+['ours-crowded']+['Benchmark']*7][0]
info_df

,colony,num_frames,start_cells,end_cells,total_cells,total_true,total_edges,tot_to_true_ratio,source
0,0,146,4,354,11973,11969,2171712,181.444732,ours-crowded
1,1,181,2,82,5223,5221,244669,46.862478,ours
2,2,181,7,155,9960,9953,886708,89.089521,ours
3,3,181,2,96,4986,4984,253895,50.942014,ours
4,4,181,2,123,5753,5751,379526,65.993045,ours
5,5,181,2,38,2722,2720,60156,22.116176,ours
6,6,120,3,331,8263,8260,1283611,155.400847,external
7,7,97,44,357,23683,23639,6649708,281.302424,marco
8,8,145,3,366,12408,12405,2345133,189.047400,ours-crowded
9,1,60,14,27,1150,1136,22256,19.591549,Benchmark


In [13]:
info_df['growth_rate'] = [calculate_growth_rate(info_df['start_cells'][i], info_df['end_cells'][i], info_df['num_frames'][i]) for i in range(len(info_df))]

In [14]:
info_df

,colony,num_frames,start_cells,end_cells,total_cells,total_true,total_edges,tot_to_true_ratio,source,growth_rate
0,0,146,4,354,11973,11969,2171712,181.444732,ours-crowded,0.030705
1,1,181,2,82,5223,5221,244669,46.862478,ours,0.020517
2,2,181,7,155,9960,9953,886708,89.089521,ours,0.017113
3,3,181,2,96,4986,4984,253895,50.942014,ours,0.021388
4,4,181,2,123,5753,5751,379526,65.993045,ours,0.022757
5,5,181,2,38,2722,2720,60156,22.116176,ours,0.016268
6,6,120,3,331,8263,8260,1283611,155.400847,external,0.039196
7,7,97,44,357,23683,23639,6649708,281.302424,marco,0.021583
8,8,145,3,366,12408,12405,2345133,189.047400,ours-crowded,0.033131
9,1,60,14,27,1150,1136,22256,19.591549,Benchmark,0.010946
